# Single-output tuning for MOFs

In [2]:
%reload_ext autoreload
%autoreload 2

import time

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from pycm import ConfusionMatrix
import wandb

from gpt3forchem.mofs.data import get_data
from gpt3forchem.mofs.baseline.classification import XGBClassificationBaseline
from gpt3forchem.mofs.constants import TARGETS, FEATURES, CAT_TARGETS, TEXT
from gpt3forchem.mofs.create_prompts import create_single_property_forward_prompts
from gpt3forchem.fine_tune import fine_tune
from gpt3forchem.query_model import query_gpt3, extract_prediction

/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load data

In [3]:
df = get_data()

In [4]:
len(df)

3375

### Small data (N=100) setting

In [5]:
train_df, test_df = train_test_split(df, train_size=100, stratify=df[CAT_TARGETS[0]])
train_size = len(train_df)
test_size = len(test_df)
print(f"{len(train_df)} train points and {len(test_df)} test points")


100 train points and 3275 test points


#### Tune and test a baseline

In [7]:
run = wandb.init(
    project="gpt-3",
    job_type="train-baseline-model",
    config={
        "train_size": train_size,
        "test_size": test_size,
        "features": FEATURES,
        "targets": [CAT_TARGETS[0]],
        "ds": "mof",
    },
)


wandb: Currently logged in as: kjappelbaum. Use `wandb login --relogin` to force relogin


In [8]:
baseline = XGBClassificationBaseline(43)


In [9]:
baseline.tune(train_df[FEATURES], train_df[CAT_TARGETS[0]])


[I 2022-06-27 09:09:41,564] A new study created in memory with name: no-name-a2b7a4b0-8ba4-4d23-adee-d091500a9e04
/Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/src/gpt3forchem/mofs/baseline/classification.py:39: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.05),
/Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/src/gpt3forchem/mofs/baseline/classification.py:40: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "colsample_bytree": trial.suggest_loguniform("colsample_bytree", 0.2, 1),
/Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/src/gpt3forchem/mofs/baseline/classi

In [ ]:
baseline.fit(train_df[FEATURES].values, train_df[CAT_TARGETS[0]])
predictions = baseline.predict(test_df[FEATURES].values)


In [ ]:
run.log(
    {
        "distribution confusion matrix": wandb.plot.confusion_matrix(
            y_true=baseline.label_encoder.transform(test_df[CAT_TARGETS[0]].values),
            preds=baseline.label_encoder.transform(predictions),
            class_names=["very small", "small", "medium", "large", "very large"],
        )
    }
)


In [ ]:
cm = ConfusionMatrix(
    test_df[CAT_TARGETS[0]].values,
    predictions,
)


In [ ]:
run.log(
    {"ACC": cm.ACC, "ACC_macro": cm.ACC_Macro, "F1_micro": cm.F1_Micro, "F1_macro": cm.F1_Macro}
)


In [ ]:
 {"ACC": cm.ACC, "ACC_macro": cm.ACC_Macro, "F1_micro": cm.F1_Micro, "F1_macro": cm.F1_Macro}

In [ ]:
run.finish()


#### Fine-tune GPT-3

In [ ]:
train_prompts = create_single_property_forward_prompts(train_df, CAT_TARGETS[0], TEXT[1])
valid_prompts = create_single_property_forward_prompts(test_df, CAT_TARGETS[0], TEXT[1])


In [ ]:
filename_base = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
train_filename = f"run_files/{filename_base}_train_prompts_mof_{train_size}.jsonl"
valid_filename = f"run_files/{filename_base}_valid_prompts_mof_{test_size}.jsonl"
# to save money, just run a small valid frame
valid_small_filename = f"run_files/{filename_base}_validsmall_prompts_mof_{test_size}.jsonl"
train_prompts.to_json(train_filename, orient="records", lines=True)
valid_prompts.to_json(valid_filename, orient="records", lines=True)
valid_prompts.sample(100).to_json(valid_small_filename, orient="records", lines=True)


In [ ]:
train_filename

In [ ]:
fine_tune(train_filename, valid_filename)